# Import Data

## Présentation
Lorsqu'il est nécessaire de mettre à jour la base de données MongoDB avec de nouvelles données, il peut être pratique de supprimer tous les documents (Tweets) de la collection existante et d'insérer les nouveaux documents à partir des fichiers JSON contenu dans le dossier ``data`` à la racine du projet pour garantir la cohérence des données et s'assurer que nous ne disposons pas de données modifiées. Dans ce cas-ci, ce script peut être utilisé.

Que fait-il ?
1. Suppression des données existantes dans la base de données MongoDB ayant pour nom Tweet
2. Lecture des données présente dans le dossier ``data`` à la racine du projet. A noter que nous n'avons pas commit ces données à cause d'une taille volumineuse. Elles peuvent être téléchargées via le lien suivant : [Cliquer ici](https://www.dropbox.com/s/qfhaobip55xxkif/Tweet%20Worldcup.zip?dl=0)
3. Insertions des données lues dans la base de données

## Le code
On importe les différentes bibliothèques nécessaires.

In [ ]:
import pymongo
import json
import os
from datetime import datetime
import numpy as np

Connexion à la base de données MongoDB

In [ ]:
client = pymongo.MongoClient("mongodb://localhost:27017")
db = client["Tweet"]
collection = db["dataset"]

Suppression de tous les documents de la collection

In [ ]:
collection.delete_many({})

In [ ]:
tweet_features_to_keep = ["created_date","id","text","entities","favorite_count","lang","possibly_sensitive","retweet_count","source","user"]
user_features_to_keep = ["friends_count","favourites_count","created_at","id","verified","followers_count","statuses_count"]
def filter_features(tweet):
    tweet_keys = list(tweet.keys())
    tweet_user_keys = list(tweet['user'].keys())
    for key in tweet_keys:
        if key not in tweet_features_to_keep:
            tweet.pop(key, None)
    for key in tweet_user_keys:
        if key not in user_features_to_keep:
            tweet['user'].pop(key, None)

Définition de la fonction pour upload dans mongodb tout les documents contenus dans un fichier

In [ ]:
def upload_file_data(path):
    with open(path, encoding='UTF-8') as f:
        docs = []
        for line in f:
            doc = json.loads(line)
            filter_features(doc)
            doc["created_date"] = datetime.strptime(doc["created_date"],"%Y-%m-%d %H:%M:%S")
            doc["user"]["created_at"] = datetime.strptime(doc["user"]["created_at"], "%a %b %d %H:%M:%S %z %Y")
            docs.append(doc)
        if len(docs) > 0:
            collection.insert_many(docs)


Récupération des noms des fichiers JSON dans le dossier approprié

In [ ]:
path = "../../data"
all_files = os.listdir(path)

On parcourt l'ensemble des fichiers pour upload les documents qui le constitue tout en affichant des statistiques pour suivre l'évolution de l'importation des données dans MongoDB

In [ ]:
start_time = datetime.now()
print(f">>>> Start : {start_time}")

# begins upload
nb_files = len(all_files)
for idx_files, file in enumerate(all_files):
    start_file_time = datetime.now() 
    if file.endswith(".json"):
        upload_file_data(f"{path}/{file}")
    time_for_this_file = (datetime.now()  - start_file_time).seconds
    percentage = round(idx_files*100/nb_files,2)
    print(f" - Upload files n°{idx_files}/{nb_files} in {time_for_this_file}s ({percentage}%)")

# some stats
end_time = datetime.now()     
print(f">>>> End : {end_time}")
diff_time = end_time - start_time
print(f"Upload done in {diff_time.seconds // 3600}h {(diff_time.seconds // 60) % 60}min {diff_time.seconds % 60}sec ")

## Suppression des doublons

On peut exécuter la cellue suivante afin d'observer si la base de données  comporte ou non des doublons. Ils sont triés par ordre décroissant du nombre total d'apparitions. C'est à dire que les premiers résultats désigne les documents qui apparaissent le plus fréquemment.

In [ ]:
res = collection.aggregate([
    {
        "$group": {
            "_id": "$id",
            "count": { "$sum": 1 }
        }
    },
    {
        "$match": {
            "count": { "$gt": 1 }
        }
    },
    { 
        "$sort" : { 
            "count" : -1 
        } 
    }
])

docs = []
for doc in res:
    print(doc)
    docs.append(doc)

if len(docs) == 0:
    print('Aucun doublon')

On supprime les doublons

In [ ]:
pipeline = [
    {"$group": {"_id": "$id", "count": {"$sum": 1}, "dups": {"$addToSet": "$_id"}}},
    {"$match": {"count": {"$gt": 1}}}
]

duplicates = []
for doc in collection.aggregate(pipeline):
    duplicates.extend(doc["dups"][1:])

collection.delete_many({"_id": {"$in": duplicates}})

## Creation d'indexs 
Afin de rendre les requêtes à suivre plus rapides on crée des indexes au sein de notre base de données mongoDB

In [ ]:
collection.create_index([("id",1)])
collection.create_index([("user.id",1)])
collection.create_index([("user.id",1),("id",1)])
collection.create_index([("user.id",1),("created_date",1)])